# Predicting Emotion and Sentiment

This notebook shows you how to take the text responses from your dialogue table and predict:

1. **Emotion labels** via our [Emotion Model](https://huggingface.co/j-hartmann/emotion-english-distilroberta-base)
2. **Sentiment scores** via our [Sentiment Model](https://huggingface.co/cardiffnlp/twitter-roberta-base-sentiment-latest)

> **Before you begin:**  
> Run all previous notebooks before proceeding.

---


In [1]:
import warnings
warnings.filterwarnings("ignore")

import torch
import numpy as np
import os
from transformers import pipeline
import glob

import sys
sys.path.append('../pyfiles/')
from dialog import get_start_end_referencedf

import pandas as pd
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 504)
pd.set_option('display.max_colwidth', None)

tempfile = "temp.wav"

---

In this example, we predict text-based emotion and sentiment for each response. Please adjust the following variables:

- `audiopath`: A string containing the path to your two‑channel audio file.  
- `feature_dir`: A string specifying the directory where all preprocessed outputs are stored.  

---


In [3]:
###########################################
########## Adjustable Parameters ##########
###########################################

audiopath = "../audio/sample.wav"
feature_dir = "../audio/features/sample/"

###########################################
###########################################
###########################################

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device {device}")

### Classifiers ###
model_path = "j-hartmann/emotion-english-distilroberta-base"
text_emotion = pipeline("text-classification", model=model_path, return_all_scores=True, device=device)

model_path = "cardiffnlp/twitter-roberta-base-sentiment-latest"
text_sentiment = pipeline("sentiment-analysis", model=model_path, tokenizer=model_path, device=device)

print("#####################")
print("### Load all data ###")
print("#####################")

resultpath = feature_dir + "whisper/" + os.path.basename(audiopath[:-4]) + f".npy"
laughpath = feature_dir + "laughs/" + os.path.basename(audiopath[:-4]) + f".npy"
tablepath = laughpath.replace("laughs", "results")

a = np.load(tablepath, allow_pickle=True).item()
rawdata, data1, data2, data3, dfci = a["rawdata"], a["data1"], a["data2"], a["data3"], a["dfci"]
dirname = feature_dir + "classification_results/"
os.makedirs(dirname, exist_ok=True)

print("##########################################################")
print("### Get classification results (emotion and sentiment) ###")
print("##########################################################")

for idx in range(len(data1)):
    arrayturn = data1.iloc[idx]
    startllm, endllm = get_start_end_referencedf(rawdata, arrayturn)
    savepath = dirname + f'classificationresults_{startllm}_{endllm}.npy'
    if os.path.exists(savepath):
        continue

    text = arrayturn["transcription"]

    # Text Classification
    result_te = text_emotion(text)
    result_ts = text_sentiment(text)

    data_dict = {"result_te": result_te, "result_ts": result_ts}
    np.save(savepath, data_dict)
    
print("##########################################")
print("##### Display Classification Results #####")
print("##########################################")

a = np.load(tablepath, allow_pickle=True).item()
rawdata, data1, data2, data3, dfci = a["rawdata"], a["data1"], a["data2"], a["data3"], a["dfci"]

sentiment_score = False
score_dir = {
    "Sentiment": {
        "positive": 10,
        "neutral": 0,
        "negative": -10,
    },
}

tt_classes = ["Sentiment", "Emotion"]
dirname = feature_dir + "classification_results/"
keys = ["_".join(os.path.basename(a[:-4]).split("_")[1:3]) for a in glob.glob(dirname+"classificationresults_*.npy")]
keys.sort()

udfst = data1.copy()
udfst["Sentence-Emotion"] = ""
udfst["Sentence-Sentiment"] = np.nan
for key in keys:
    # Obtain the prediction results
    path = dirname + f"classificationresults_{key}.npy"
    hey = np.load(path, allow_pickle=True).item()
    result_te, result_ts = hey["result_te"], hey["result_ts"]
    a = [a["label"] for a in result_te[0]]
    b = [a["score"] for a in result_te[0]]
    result_te = a[np.argmax(b)]
    result_ts = result_ts[0]["label"]

    # Insert the information to udfst
    startllm, endllm = [int(a) for a in key.split("_")]
    try:
        start = np.arange(len(udfst))[np.abs(rawdata.iloc[startllm]["start"]-udfst["start"])<1e-5]
        end = np.arange(len(udfst))[np.abs(rawdata.iloc[endllm]["end"]-udfst["end"])<1e-5]
        idx = list(set(list([udfst.iloc[a].name for a in start])) & set(list([udfst.iloc[a].name for a in end])))[0]
        udfst.loc[idx, "Sentence-Emotion"] = result_te
        udfst.loc[idx, "Sentence-Sentiment"] = score_dir["Sentiment"][result_ts]
        if not(sentiment_score):
            udfst.loc[udfst["Sentence-Sentiment"]==10, "Sentence-Sentiment"] = "positive"
            udfst.loc[udfst["Sentence-Sentiment"]==0, "Sentence-Sentiment"] = "neutral"
            udfst.loc[udfst["Sentence-Sentiment"]==-10, "Sentence-Sentiment"] = "negative"
    except IndexError:
        continue
udfst

Using device cuda


Some weights of the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment-latest were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


#####################
### Load all data ###
#####################
##########################################################
### Get classification results (emotion and sentiment) ###
##########################################################
##########################################
##### Display Classification Results #####
##########################################


,start,end,speaker,transcription,duration,duration-before-talking,Overlap,Fully-Overlap,start-timeshift,end-timeshift,duration-before-talking-timeshift,Overlap-timeshift,Fully-Overlap-timeshift,sentence start,Sentence-Emotion,Sentence-Sentiment
0,1.720,3.280000,A,hello hi this is dina,1.560000,0.000000,,False,1.720,3.280000,0.000000,,False,0,neutral,neutral
1,3.972,4.396000,B,hi i am donna,0.424000,0.000000,,False,3.972,4.396000,0.000000,,False,,surprise,neutral
2,4.908,5.060000,A,hi,0.152000,1.628000,,False,4.908,5.060000,1.628000,,False,,neutral,neutral
3,6.312,6.980000,A,so do you have pets,0.668000,1.252000,,False,6.312,6.980000,1.252000,,False,,neutral,neutral
4,8.056,8.500000,B,uh no,0.444000,3.660000,,False,8.056,8.500000,3.660000,,False,,neutral,neutral
5,8.972,9.180000,A,oh,0.208000,1.992000,,False,8.972,9.180000,1.992000,,False,,neutral,neutral
6,10.072,10.600000,B,to you,0.528000,1.572000,,False,10.072,10.600000,1.572000,,False,,neutral,neutral
7,10.960,15.200000,A,yeah i do i have [StartLaugh] three dogs and [EndLaugh] i have a bunch of fish,4.240000,1.780000,,False,10.960,15.200000,1.780000,,False,,neutral,neutral
8,16.004,16.300062,B,uh oh,0.296062,5.404000,,False,16.004,16.300062,5.404000,,False,,neutral,neutral
9,16.584,18.220000,A,i have a black lab,1.636000,1.384000,,False,16.584,18.220000,1.384000,,False,,sadness,neutral
